<h4>Follow steps along numbers</h4>

#1 Variables <Br>
#2 Style init<Br>
#3 Context init (Three options)<Br>
#4 Main<Br>

In [ ]:
#1 Variables
MODEL_NAME = "gpt-3.5-turbo"
TEMPERATURE = 0
OPENAI_API_KEY = ""
import os
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
#######################
STYLE_FILE = "data/conf_styles.pickle"

STYLE_PROMPT = """
당신은 현대 언어를 공자처럼 바꿔주는 전문 번역가입니다. 
다음 절차를 따라 저의 말을 번역해주세요.

첫째, 실제 공자가 이야기하는 듯한 느낌을 주기 위하여 1인칭으로 바꾸어 말합니다. 
둘째, 공자가 말하듯이 공자의 말을 인용한다는 부분 또는 논어를 참고한다는 부분을 삭제합니다. 
셋째, 하오체를 사용합니다. 
"""
#######################
DATA_FILE = "data/analects_raw.txt"
SPLIT_CHUNK_SIZE = 500
SPLIT_CHUNK_OVERLAP = 10

CONTEXT_PROMPT = """
당신은 'conf'를 바탕으로 답변을 만드는 챗봇입니다.
다음 절차를 따라 저의 질문에 답변해 주세요.

첫째, 질문에 답변하기 위해 'conf'에서 근거를 찾아 '근거:' 뒤에 출력하세요. 사용자의 질문과 조금이라도 관련되었다면 모두 근거로 사용하세요.
만약 'conf'에 근거가 없다면 '없음'이라고 적고, 공자가 했을 뻡한 말을 답변: 뒤에 출력하세요.
둘째, 근거가 있다면 해당 근거를 바탕으로 '답변:' 뒤에 답변을 출력하세요.

근거: 여기에 근거를 출력
답변: 여기에 답변을 출력
"""

In [ ]:
#2 Style
import os
from langchain.prompts import PromptTemplate, FewShotPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
import pickle

def style_init() -> LLMChain:
    example_prompt = PromptTemplate(
        input_variables=["input", "output"],
        template="Input: {input}\nOutput: {output}",
    )
    with open(STYLE_FILE, "rb") as pr:
        examples = pickle.load(pr)

    example_selector = SemanticSimilarityExampleSelector.from_examples(
        examples, OpenAIEmbeddings(), Chroma, k=10
    )
    similar_prompt = FewShotPromptTemplate(
        example_selector=example_selector,
        example_prompt=example_prompt,
        prefix= STYLE_PROMPT,
        suffix="Input: {input_noun}\n",
        input_variables=["input_noun"],
    )
    chat_model = ChatOpenAI(temperature=TEMPERATURE, model=MODEL_NAME)
    chain = LLMChain(llm=chat_model, prompt=similar_prompt, verbose=True)
    return chain

style_chain = style_init()

In [ ]:
#3-1 Only GPT
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI

def context_init() -> LLMChain:
    prompt = PromptTemplate(input_variables=["in"],
    template="""
    너는 공자야.
    input:{in}
    """)
    chat_model = ChatOpenAI(temperature=TEMPERATURE, model=MODEL_NAME)
    chain = LLMChain(llm=chat_model,prompt=prompt, verbose=True)
    return chain

context_chain = context_init()
num = 1

In [ ]:
#3-2 Data (ChatVectorDBChain)
from langchain.document_loaders import TextLoader
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import ChatVectorDBChain

def context_init() -> ChatVectorDBChain:
    loader = TextLoader(DATA_FILE, encoding='utf-8')
    data = loader.load()
    text_splitter = CharacterTextSplitter(
        separator="\n\n\n",
        chunk_size=SPLIT_CHUNK_SIZE, 
        chunk_overlap=SPLIT_CHUNK_OVERLAP
    )
    splits = text_splitter.split_documents(data)
    
    #vectordb
    embedding = HuggingFaceEmbeddings()
    vectordb = FAISS.from_documents(
        documents = splits,
        embedding = embedding
    )
    
    model=ChatOpenAI(temperature=TEMPERATURE,model_name=MODEL_NAME)
    chain=ChatVectorDBChain.from_llm(model,vectordb,return_source_documents=True, verbose=True)
    return chain

context_chain = context_init()
num = 2

In [ ]:
#3-3 Data + Prompt (agent)
from langchain.tools import Tool
from langchain.document_loaders import TextLoader
from langchain.memory import ConversationBufferMemory
from langchain.agents.openai_functions_agent.base import OpenAIFunctionsAgent
from langchain.schema import SystemMessage
from langchain.prompts import MessagesPlaceholder
from langchain.chat_models import ChatOpenAI
from langchain.agents import AgentExecutor
from langchain.chains import RetrievalQA
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS


def context_init() -> AgentExecutor:
    # RetrievalQA
    loader = TextLoader(DATA_FILE, encoding='utf-8')
    data = loader.load()
    text_splitter = CharacterTextSplitter(
        separator="\n\n\n",
        chunk_size=SPLIT_CHUNK_SIZE, 
        chunk_overlap=SPLIT_CHUNK_OVERLAP
    )
    splits = text_splitter.split_documents(data)
    
    #vectordb
    embedding = HuggingFaceEmbeddings()
    vectordb = FAISS.from_documents(
        documents = splits,
        embedding = embedding
    )
    llm = ChatOpenAI(model=MODEL_NAME, temperature=TEMPERATURE)
    retrievalQA_chain = RetrievalQA.from_chain_type(llm=llm, chain_type='stuff', retriever=vectordb.as_retriever())
    personal_data_tool = Tool(name='conf', func=retrievalQA_chain.run, description='논어 데이터')
    # agent
    memory_key = 'history'
    memory = ConversationBufferMemory(memory_key=memory_key, return_messages=True)
    system_message = SystemMessage(content=CONTEXT_PROMPT)
    prompt = OpenAIFunctionsAgent.create_prompt(system_message=system_message, extra_prompt_messages=[MessagesPlaceholder(variable_name=memory_key)])
    agent = OpenAIFunctionsAgent(llm=llm, tools=[personal_data_tool], prompt=prompt)
    agent_executor = AgentExecutor(agent=agent, tools=[personal_data_tool], memory=memory, verbose=True, handle_parsing_errors=True)
    return agent_executor

context_chain = context_init()
num = 3


In [ ]:
#4 Main
def get_user_input():
    return input('\033[31m\r\nPrompt: \033[0m')

def main():
    user_input = get_user_input()
    if num == 1:
        result=context_chain(inputs=user_input)
        conf_result = style_chain(inputs=result['text'])
    elif num == 2:
        result=context_chain({'question':user_input,'chat_history':[]})
        conf_result = style_chain(inputs=result['answer'])
    elif num == 3:
        result = context_chain({'input': user_input}, return_only_outputs=True)
        conf_result = style_chain(inputs=result['output'])
    
    print(conf_result["text"])

if __name__ == "__main__":
    main()